In [1]:
import pandas as pd

data = {
    "time": [
        "00:00","01:00","02:00","03:00","04:00","05:00","06:00","07:00","08:00","09:00",
        "10:00","11:00","12:00","13:00","14:00","15:00","16:00","17:00","18:00","19:00",
        "20:00","21:00","22:00","23:00"
    ],
    "solar_kW": [0,0,0,0,0,10,50,120,200,300,400,450,480,460,400,300,200,100,50,10,0,0,0,0],
    "wind_kW": [20,15,10,5,10,20,30,40,50,60,80,100,120,130,100,80,70,60,50,40,30,25,20,15],
    "demand_kW": [50,45,40,35,40,60,80,100,120,150,200,220,230,220,210,200,180,150,120,100,90,80,70,60],
    "price_buy": [6]*24,
    "price_sell": [4]*24
}

df = pd.DataFrame(data)

# Save in your system (change path if needed)
df.to_csv(r"H:\\Hackout\\Mangrove\\energy_data_1000.csv", index=False)

print("✅ File saved at: H:\\Hackout\\Mangrove\\energy_data_1000.csv")


✅ File saved at: H:\Hackout\Mangrove\energy_data_1000.csv


In [3]:
import cv2
import numpy as np
import json
from datetime import datetime
from shapely.geometry import Point
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import geopandas as gpd

def convert_numpy_types(obj):
    """Convert numpy types to JSON serializable types"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {k: convert_numpy_types(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif hasattr(obj, 'item'):
        return obj.item()
    return obj

def extract_gps_from_image(image_path):
    """Extract GPS coordinates from image EXIF data"""
    try:
        image = Image.open(image_path)
        exif_data = image._getexif()
        
        if exif_data is None:
            return None, "No EXIF data found"
        
        gps_info = {}
        for tag, value in exif_data.items():
            tag_name = TAGS.get(tag, tag)
            if tag_name == "GPSInfo":
                for gps_tag in value:
                    sub_tag_name = GPSTAGS.get(gps_tag, gps_tag)
                    gps_info[sub_tag_name] = value[gps_tag]
        
        if not gps_info:
            return None, "No GPS data in image"
        
        lat = convert_to_degrees(gps_info.get('GPSLatitude'))
        lng = convert_to_degrees(gps_info.get('GPSLongitude'))
        
        if gps_info.get('GPSLatitudeRef') == 'S':
            lat = -lat
        if gps_info.get('GPSLongitudeRef') == 'W':
            lng = -lng
            
        return (lat, lng), "GPS extracted successfully"
        
    except Exception as e:
        return None, f"Error extracting GPS: {str(e)}"

def convert_to_degrees(gps_coordinate):
    """Convert GPS coordinate to decimal degrees"""
    if gps_coordinate is None:
        return None
    degrees = float(gps_coordinate[0])
    minutes = float(gps_coordinate[1])
    seconds = float(gps_coordinate[2])
    return degrees + (minutes / 60.0) + (seconds / 3600.0)

class MangroveClassifier:
    """Model 1: Classify if image contains mangroves or not"""
    
    def __init__(self):
        pass
    
    def detect_vegetation(self, hsv, image):
        """Detect all vegetation including sparse/damaged areas"""
        h, w = image.shape[:2]
        
        # Healthy green vegetation
        lower_green1 = np.array([35, 30, 30])   # Lowered saturation threshold
        upper_green1 = np.array([85, 255, 255])
        green_mask1 = cv2.inRange(hsv, lower_green1, upper_green1)
        
        # Darker/stressed green vegetation
        lower_green2 = np.array([40, 40, 20])   # Lower brightness for stressed plants
        upper_green2 = np.array([80, 255, 180])
        green_mask2 = cv2.inRange(hsv, lower_green2, upper_green2)
        
        # Yellowish vegetation (stressed/dying)
        lower_yellow = np.array([20, 30, 40])
        upper_yellow = np.array([40, 180, 200])
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Combine all vegetation types
        vegetation_mask = cv2.bitwise_or(green_mask1, green_mask2)
        vegetation_mask = cv2.bitwise_or(vegetation_mask, yellow_mask)

        lower_green3 = np.array([30, 25, 25])   # Very inclusive for mangrove greens
        upper_green3 = np.array([90, 255, 255])
        green_mask3 = cv2.inRange(hsv, lower_green3, upper_green3)
        
        # Update the combination line to:
        vegetation_mask = cv2.bitwise_or(green_mask1, green_mask2)
        vegetation_mask = cv2.bitwise_or(vegetation_mask, green_mask3)  # Add this line
        vegetation_mask = cv2.bitwise_or(vegetation_mask, yellow_mask)
        
        # Clean up noise but preserve sparse vegetation
        kernel = np.ones((2,2), np.uint8)  # Smaller kernel
        vegetation_mask = cv2.morphologyEx(vegetation_mask, cv2.MORPH_CLOSE, kernel)
        
        vegetation_percent = (np.sum(vegetation_mask > 0) / (h * w)) * 100
        return vegetation_percent, vegetation_mask

    def detect_coastal_features(self, hsv, image):
        """Detect features specific to coastal environments"""
        h, w = image.shape[:2]
        
        # Sand/mudflats
        lower_sand = np.array([10, 15, 100])
        upper_sand = np.array([40, 100, 255])
        sand_mask = cv2.inRange(hsv, lower_sand, upper_sand)
        
        # Tidal pools/saltwater (distinctive blue-green)
        lower_tidal = np.array([85, 60, 60])
        upper_tidal = np.array([110, 255, 200])
        tidal_mask = cv2.inRange(hsv, lower_tidal, upper_tidal)
        
        coastal_features = cv2.bitwise_or(sand_mask, tidal_mask)
        coastal_percent = (np.sum(coastal_features > 0) / (h * w)) * 100
        
        return coastal_percent

    def detect_water_presence(self, hsv, image):
        """Enhanced water detection specific to mangrove environments"""
        h, w = image.shape[:2]
        
        # Only detect actual blue water (not shadows)
        lower_water1 = np.array([85, 30, 30])    # More inclusive blue water
        upper_water1 = np.array([130, 255, 255])
        # lower_water1 = np.array([100, 50, 50])  # More restrictive blue
        # upper_water1 = np.array([130, 255, 200])
        water_mask1 = cv2.inRange(hsv, lower_water1, upper_water1)
        
        # Muddy/brackish water specific to mangroves
        lower_water2 = np.array([8, 20, 40])     # More inclusive muddy water  
        upper_water2 = np.array([40, 200, 180])
        # lower_water2 = np.array([15, 40, 60])   # More restrictive muddy water
        # upper_water2 = np.array([35, 180, 140])
        water_mask2 = cv2.inRange(hsv, lower_water2, upper_water2)
        
        water_features = cv2.bitwise_or(water_mask1, water_mask2)
        water_percent = (np.sum(water_features > 0) / (h * w)) * 100
        return water_percent
    
    
    def detect_mangrove_indicators(self, hsv, image):
        """Detect mangrove-specific features even in damaged areas"""
        h, w = image.shape[:2]
        
        # Mangrove root structures (aerial roots, prop roots)
        lower_roots = np.array([8, 20, 30])
        upper_roots = np.array([35, 150, 120])
        root_mask = cv2.inRange(hsv, lower_roots, upper_roots)
        
        # Tree trunks and branches (even dead ones)
        lower_trunk = np.array([5, 15, 40])
        upper_trunk = np.array([30, 100, 160])
        trunk_mask = cv2.inRange(hsv, lower_trunk, upper_trunk)
        
        # Mangrove-like structure indicators
        structure_mask = cv2.bitwise_or(root_mask, trunk_mask)
        structure_percent = (np.sum(structure_mask > 0) / (h * w)) * 100
        
        return structure_percent
    
    def classify_mangrove(self, image_path):
        """Enhanced classification for damaged mangroves"""
        
        image = cv2.imread(image_path)
        if image is None:
            return False, {"error": "Could not load image"}
        
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        vegetation_percent, vegetation_mask = self.detect_vegetation(hsv, image)
        water_percent = self.detect_water_presence(hsv, image)
        structure_percent = self.detect_mangrove_indicators(hsv, image)
        
        # Enhanced classification logic for damaged areas
        mangrove_confidence = 0
        
        # Coastal features check (keep existing)
        coastal_percent = self.detect_coastal_features(hsv, image)
        
        # More lenient water requirements
        if water_percent > 8:
            mangrove_confidence += 40
        elif water_percent > 3:
            mangrove_confidence += 25
        elif water_percent > 1:
            mangrove_confidence += 15
        
        # More lenient vegetation requirements  
        if vegetation_percent > 15:
            mangrove_confidence += 30
        elif vegetation_percent > 8:
            mangrove_confidence += 20
        elif vegetation_percent > 3:
            mangrove_confidence += 10
        
        # Structure detection (keep existing logic)
        if structure_percent > 10:
            mangrove_confidence += 20
        elif structure_percent > 5:
            mangrove_confidence += 15
        elif structure_percent > 2:
            mangrove_confidence += 10
        
        # Coastal bonus
        if coastal_percent > 3:
            mangrove_confidence += 15
        
        # Lower the threshold significantly
        is_mangrove = mangrove_confidence > 45  # Reduced from 65
        
        return is_mangrove, {
            "is_mangrove": is_mangrove,
            "confidence": round(mangrove_confidence, 1),
            "vegetation_percent": round(vegetation_percent, 1),
            "water_percent": round(water_percent, 1),
            "structure_percent": round(structure_percent, 1),
            "likely_damaged": vegetation_percent < 15 and structure_percent > 5
        }


class EnhancedMangroveDamageDetector:
    """Enhanced Model 2: Comprehensive mangrove health and damage analysis"""
    
    def __init__(self):
        pass
    
    def detect_healthy_vegetation(self, hsv, image):
        """Detect healthy green vegetation"""
        h, w = image.shape[:2]
        
        # Vibrant healthy green
        lower_healthy = np.array([40, 80, 80])
        upper_healthy = np.array([80, 255, 255])
        healthy_mask = cv2.inRange(hsv, lower_healthy, upper_healthy)
        
        # Dense green foliage
        lower_dense = np.array([35, 100, 100])
        upper_dense = np.array([75, 255, 200])
        dense_mask = cv2.inRange(hsv, lower_dense, upper_dense)
        
        healthy_vegetation = cv2.bitwise_or(healthy_mask, dense_mask)
        healthy_percent = (np.sum(healthy_vegetation > 0) / (h * w)) * 100
        
        return healthy_percent, healthy_vegetation
    
    def detect_cutting_damage(self, hsv, image):
        """Enhanced detection of cutting/clear-cutting damage"""
        h, w = image.shape[:2]
        
        # Dead stumps and cut trunks (gray/brown)
        lower_stump = np.array([5, 10, 40])    # Expanded range for dead wood
        upper_stump = np.array([35, 120, 160])
        stump_mask = cv2.inRange(hsv, lower_stump, upper_stump)
        
        # Exposed dead wood/bark (lighter browns/grays)
        lower_wood = np.array([8, 20, 60])
        upper_wood = np.array([25, 80, 180])
        wood_mask = cv2.inRange(hsv, lower_wood, upper_wood)
        
        # Dead root systems (characteristic white/gray color)
        lower_roots = np.array([0, 0, 100])    # Light colored dead roots
        upper_roots = np.array([30, 60, 200])
        dead_roots_mask = cv2.inRange(hsv, lower_roots, upper_roots)
        
        # Very light/bleached dead material
        lower_bleached = np.array([0, 0, 150])
        upper_bleached = np.array([180, 40, 255])
        bleached_mask = cv2.inRange(hsv, lower_bleached, upper_bleached)
        
        # Combine all cutting indicators
        cutting_mask = cv2.bitwise_or(stump_mask, wood_mask)
        cutting_mask = cv2.bitwise_or(cutting_mask, dead_roots_mask)
        cutting_mask = cv2.bitwise_or(cutting_mask, bleached_mask)
        
        cutting_percent = (np.sum(cutting_mask > 0) / (h * w)) * 100
        
        return cutting_percent, cutting_mask
    
    def detect_natural_damage(self, hsv, image):
        """Detect natural damage (disease, storms, etc.)"""
        h, w = image.shape[:2]
        
        # Yellowing/browning leaves (disease/stress)
        lower_yellow = np.array([15, 100, 100])
        upper_yellow = np.array([35, 255, 200])
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        # Dead/dried vegetation
        lower_dead = np.array([10, 20, 40])
        upper_dead = np.array([25, 100, 120])
        dead_mask = cv2.inRange(hsv, lower_dead, upper_dead)
        
        natural_damage = cv2.bitwise_or(yellow_mask, dead_mask)
        natural_damage_percent = (np.sum(natural_damage > 0) / (h * w)) * 100
        
        return natural_damage_percent, natural_damage
    def detect_pollution_damage(self, hsv, image):
        """Detect pollution indicators like oil spills, discolored water"""
        h, w = image.shape[:2]
        
        # Oil slicks (dark iridescent surfaces)
        lower_oil = np.array([0, 30, 30])
        upper_oil = np.array([20, 150, 100])
        oil_mask = cv2.inRange(hsv, lower_oil, upper_oil)
        
        # Discolored/polluted water
        lower_polluted = np.array([40, 80, 80])
        upper_polluted = np.array([70, 255, 150])
        polluted_mask = cv2.inRange(hsv, lower_polluted, upper_polluted)
        
        pollution_mask = cv2.bitwise_or(oil_mask, polluted_mask)
        pollution_percent = (np.sum(pollution_mask > 0) / (h * w)) * 100
        
        return pollution_percent, pollution_mask
    
    def detect_climate_stress(self, hsv, image):
        """Detect climate-related stress (salinity, temperature)"""
        h, w = image.shape[:2]
        
        # Salt-stressed vegetation (pale/whitish leaves)
        lower_salt = np.array([45, 20, 120])
        upper_salt = np.array([75, 100, 255])
        salt_stress_mask = cv2.inRange(hsv, lower_salt, upper_salt)
        
        # Heat-stressed vegetation (reddish-brown)
        lower_heat = np.array([0, 80, 80])
        upper_heat = np.array([15, 255, 180])
        heat_stress_mask = cv2.inRange(hsv, lower_heat, upper_heat)
        
        climate_stress = cv2.bitwise_or(salt_stress_mask, heat_stress_mask)
        climate_stress_percent = (np.sum(climate_stress > 0) / (h * w)) * 100
        
        return climate_stress_percent, climate_stress
    
    def calculate_canopy_density(self, vegetation_mask, image):
        """Calculate vegetation density and distribution"""
        h, w = image.shape[:2]
        
        # Divide image into grid to check distribution
        grid_size = 10
        cell_h, cell_w = h // grid_size, w // grid_size
        
        cells_with_vegetation = 0
        total_cells = grid_size * grid_size
        
        for i in range(grid_size):
            for j in range(grid_size):
                y1, y2 = i * cell_h, (i + 1) * cell_h
                x1, x2 = j * cell_w, (j + 1) * cell_w
                
                cell_vegetation = vegetation_mask[y1:y2, x1:x2]
                if np.sum(cell_vegetation > 0) > (cell_h * cell_w * 0.1):  # 10% threshold
                    cells_with_vegetation += 1
        
        distribution_score = (cells_with_vegetation / total_cells) * 100
        return distribution_score
    
    def analyze_comprehensive_health(self, image_path):
        """Comprehensive mangrove health analysis"""
        
        image = cv2.imread(image_path)
        if image is None:
            return {"error": "Could not load image"}
        
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, w = image.shape[:2]
        
        # Get all vegetation types
        healthy_percent, healthy_mask = self.detect_healthy_vegetation(hsv, image)
        cutting_percent, cutting_mask = self.detect_cutting_damage(hsv, image)
        natural_damage_percent, natural_mask = self.detect_natural_damage(hsv, image)
        
        # Calculate canopy metrics
        all_vegetation = cv2.bitwise_or(healthy_mask, cv2.bitwise_or(cutting_mask, natural_mask))
        total_vegetation = (np.sum(all_vegetation > 0) / (h * w)) * 100
        canopy_distribution = self.calculate_canopy_density(all_vegetation, image)
        
        # Determine health status
        if healthy_percent > 40 and cutting_percent < 5 and natural_damage_percent < 10:
            health_status = "HEALTHY"
            health_score = min(95, 70 + healthy_percent * 0.5)
        elif healthy_percent > 20 and cutting_percent < 15:
            health_status = "MODERATELY_HEALTHY"
            health_score = 50 + healthy_percent * 0.8 - cutting_percent * 2
        elif cutting_percent > 25:
            health_status = "SEVERELY_CUT"
            health_score = max(10, 40 - cutting_percent * 1.5)
        elif natural_damage_percent > 30:
            health_status = "NATURALLY_DAMAGED"
            health_score = max(15, 50 - natural_damage_percent * 1.2)
        else:
            health_status = "DEGRADED"
            health_score = 30
        
        
        # Determine primary damage type and severity
        damage_types = []
        if cutting_percent > 10:
            if cutting_percent > 30:
                damage_types.append(f"Severe cutting/logging ({cutting_percent:.1f}% of area)")
            elif cutting_percent > 15:
                damage_types.append(f"Moderate cutting ({cutting_percent:.1f}% of area)")
            else:
                damage_types.append(f"Minor cutting detected ({cutting_percent:.1f}% of area)")
        
        if natural_damage_percent > 15:
            if natural_damage_percent > 35:
                damage_types.append(f"Severe natural damage ({natural_damage_percent:.1f}% of area)")
            else:
                damage_types.append(f"Natural stress/disease ({natural_damage_percent:.1f}% of area)")
        
        # Calculate estimated area affected (assuming 1 hectare reference)
        estimated_total_area = 1.0  # hectare
        healthy_area = (healthy_percent / 100) * estimated_total_area
        damaged_area = ((cutting_percent + natural_damage_percent) / 100) * estimated_total_area
        
        # Generate recommendations
        recommendations = self.generate_recommendations(health_status, cutting_percent, natural_damage_percent, canopy_distribution)
        
        # Calculate recovery timeline
        recovery_time = self.estimate_recovery_time(cutting_percent, natural_damage_percent, health_status)
        
        return {
            "overall_health": {
                "status": health_status,
                "health_score": round(health_score, 1),
                "description": self.get_health_description(health_status, health_score)
            },
            "vegetation_analysis": {
                "healthy_vegetation": round(healthy_percent, 1),
                "total_vegetation_cover": round(total_vegetation, 1),
                "canopy_distribution": round(canopy_distribution, 1)
            },
            "damage_assessment": {
                "cutting_damage": round(cutting_percent, 1),
                "natural_damage": round(natural_damage_percent, 1),
                "primary_damage_types": damage_types,
                "total_damage_score": round(cutting_percent + natural_damage_percent, 1)
            },
            "area_estimates": {
                "estimated_healthy_area_hectares": round(healthy_area, 2),
                "estimated_damaged_area_hectares": round(damaged_area, 2),
                "estimated_total_area_hectares": estimated_total_area
            },
            "conservation_status": {
                "requires_immediate_action": cutting_percent > 20 or natural_damage_percent > 30,
                "alert_authorities": cutting_percent > 15,
                "monitoring_required": health_score < 60,
                "recovery_timeline": recovery_time
            },
            "recommendations": recommendations,
            "analysis_timestamp": datetime.now().isoformat()
        }
    
    def get_health_description(self, status, score):
        """Get human-readable health description"""
        descriptions = {
            "HEALTHY": f"Thriving mangrove ecosystem with dense, healthy vegetation (Score: {score}/100)",
            "MODERATELY_HEALTHY": f"Generally healthy but showing some stress signs (Score: {score}/100)",
            "DEGRADED": f"Ecosystem showing signs of decline and stress (Score: {score}/100)",
            "SEVERELY_CUT": f"Significant human intervention/cutting detected (Score: {score}/100)",
            "NATURALLY_DAMAGED": f"Natural damage from disease, storms, or environmental stress (Score: {score}/100)"
        }
        return descriptions.get(status, f"Health status unclear (Score: {score}/100)")
    
    def analyze_comprehensive_health(self, image_path):
        """Comprehensive mangrove health analysis with enhanced damage detection"""
        
        image = cv2.imread(image_path)
        if image is None:
            return {"error": "Could not load image"}
        
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, w = image.shape[:2]
        
        # Get all vegetation and damage types
        healthy_percent, healthy_mask = self.detect_healthy_vegetation(hsv, image)
        cutting_percent, cutting_mask = self.detect_cutting_damage(hsv, image)
        natural_damage_percent, natural_mask = self.detect_natural_damage(hsv, image)
        
        # Calculate canopy metrics
        all_vegetation = cv2.bitwise_or(healthy_mask, cv2.bitwise_or(cutting_mask, natural_mask))
        total_vegetation = (np.sum(all_vegetation > 0) / (h * w)) * 100
        canopy_distribution = self.calculate_canopy_density(all_vegetation, image)
        
        # Enhanced damage scoring for severe cases
        total_damage_score = cutting_percent + natural_damage_percent
        
        # Special logic for clear-cut/severely damaged areas
        if cutting_percent > 40 or (cutting_percent > 20 and healthy_percent < 5):
            health_status = "CLEAR_CUT"
            health_score = max(5, 20 - cutting_percent * 0.3)
        elif cutting_percent > 25:
            health_status = "SEVERELY_CUT"
            health_score = max(10, 30 - cutting_percent * 0.5)
        elif cutting_percent > 15:
            health_status = "HEAVILY_CUT"
            health_score = max(20, 40 - cutting_percent * 0.8)
        elif healthy_percent > 40 and cutting_percent < 5 and natural_damage_percent < 10:
            health_status = "HEALTHY"
            health_score = min(95, 70 + healthy_percent * 0.5)
        elif healthy_percent > 20 and cutting_percent < 15:
            health_status = "MODERATELY_HEALTHY"
            health_score = 50 + healthy_percent * 0.8 - cutting_percent * 2
        elif natural_damage_percent > 30:
            health_status = "NATURALLY_DAMAGED"
            health_score = max(15, 50 - natural_damage_percent * 1.2)
        else:
            health_status = "DEGRADED"
            health_score = max(15, 40 - total_damage_score * 0.8)
        
        # Enhanced damage type classification
        damage_types = []
        
        if cutting_percent > 40:
            damage_types.append(f"🪓 CLEAR-CUTTING: Extensive removal ({cutting_percent:.1f}% of area)")
        elif cutting_percent > 25:
            damage_types.append(f"🪓 SEVERE CUTTING: Heavy logging activity ({cutting_percent:.1f}% of area)")
        elif cutting_percent > 15:
            damage_types.append(f"🪓 MODERATE CUTTING: Selective logging ({cutting_percent:.1f}% of area)")
        elif cutting_percent > 8:
            damage_types.append(f"🪓 MINOR CUTTING: Limited removal ({cutting_percent:.1f}% of area)")
        
        if natural_damage_percent > 25:
            damage_types.append(f"🌪️ SEVERE NATURAL DAMAGE: Storm/disease impact ({natural_damage_percent:.1f}% of area)")
        elif natural_damage_percent > 15:
            damage_types.append(f"🌪️ NATURAL STRESS: Environmental damage ({natural_damage_percent:.1f}% of area)")
        
        # If no vegetation but structure present - likely clear cut
        if healthy_percent < 2 and cutting_percent > 15:
            damage_types.append("💀 ECOSYSTEM COLLAPSE: Minimal living vegetation remaining")
        
        # Calculate more accurate area estimates
        estimated_total_area = 1.0  # hectare
        healthy_area = (healthy_percent / 100) * estimated_total_area
        cutting_area = (cutting_percent / 100) * estimated_total_area
        natural_damage_area = (natural_damage_percent / 100) * estimated_total_area
        
        # Generate enhanced recommendations
        recommendations = self.generate_enhanced_recommendations(health_status, cutting_percent, natural_damage_percent, canopy_distribution, healthy_percent)
        
        # Calculate recovery timeline
        recovery_time = self.estimate_enhanced_recovery_time(cutting_percent, natural_damage_percent, health_status, healthy_percent)
        
        return {
            "overall_health": {
                "status": health_status,
                "health_score": round(health_score, 1),
                "description": self.get_enhanced_health_description(health_status, health_score, cutting_percent)
            },
            "vegetation_analysis": {
                "healthy_vegetation": round(healthy_percent, 1),
                "total_vegetation_cover": round(total_vegetation, 1),
                "canopy_distribution": round(canopy_distribution, 1),
                "vegetation_loss": round(100 - total_vegetation, 1)
            },
           "damage_assessment": {
                "cutting_damage": round(cutting_percent, 1),
                "natural_damage": round(natural_damage_percent, 1),
                "primary_damage_types": damage_types,
                "total_damage_score": round(cutting_percent + natural_damage_percent, 1)
            },
                        # "damage_assessment": {
            #     "cutting_damage": round(cutting_percent, 1),
            #     "natural_damage": round(natural_damage_percent, 1),
            #     "primary_damage_types": damage_types,
            #     "total_damage_score": round(total_damage_score, 1),
            #     "severity_level": self.get_damage_severity(cutting_percent, natural_damage_percent)
            # },
            # Around line 330-340, in the return statement of analyze_comprehensive_health():
            "area_estimates": {
                "estimated_healthy_area_hectares": round(healthy_area, 3),
                "estimated_cutting_area_hectares": round(cutting_area, 3),
                "estimated_natural_damage_area_hectares": round(natural_damage_area, 3),
                "estimated_damaged_area_hectares": round(cutting_area + natural_damage_area, 3),  # ADD THIS LINE
                "estimated_total_area_hectares": estimated_total_area
            },
            # "area_estimates": {
            #     "estimated_healthy_area_hectares": round(healthy_area, 3),
            #     "estimated_cutting_area_hectares": round(cutting_area, 3),
            #     "estimated_natural_damage_area_hectares": round(natural_damage_area, 3),
            #     "estimated_total_area_hectares": estimated_total_area
            # },
            "conservation_status": {
                "requires_immediate_action": cutting_percent > 15 or natural_damage_percent > 25,
                "alert_authorities": cutting_percent > 10,
                "monitoring_required": health_score < 70,
                "recovery_timeline": recovery_time,
                "legal_action_recommended": cutting_percent > 20
            },
            "recommendations": recommendations,
            "analysis_timestamp": datetime.now().isoformat()
        }
    
    def get_damage_severity(self, cutting_percent, natural_damage_percent):
        """Determine overall damage severity"""
        total_damage = cutting_percent + natural_damage_percent
        
        if total_damage > 60 or cutting_percent > 40:
            return "CATASTROPHIC"
        elif total_damage > 40 or cutting_percent > 25:
            return "SEVERE"
        elif total_damage > 20 or cutting_percent > 15:
            return "MODERATE"
        elif total_damage > 10:
            return "MINOR"
        else:
            return "MINIMAL"
    
    def get_enhanced_health_description(self, status, score, cutting_percent):
        """Enhanced health descriptions"""
        descriptions = {
            "HEALTHY": f"Thriving mangrove ecosystem with dense, healthy vegetation (Score: {score}/100)",
            "MODERATELY_HEALTHY": f"Generally healthy but showing some stress signs (Score: {score}/100)",
            "DEGRADED": f"Ecosystem showing signs of decline and stress (Score: {score}/100)",
            "HEAVILY_CUT": f"Significant cutting damage detected - {cutting_percent:.1f}% of area affected (Score: {score}/100)",
            "SEVERELY_CUT": f"Extensive logging/cutting damage - ecosystem severely compromised (Score: {score}/100)",
            "CLEAR_CUT": f"CLEAR-CUT AREA: Massive destruction detected - {cutting_percent:.1f}% affected (Score: {score}/100)",
            "NATURALLY_DAMAGED": f"Natural damage from disease, storms, or environmental stress (Score: {score}/100)"
        }
        return descriptions.get(status, f"Health status unclear (Score: {score}/100)")
    
    def generate_enhanced_recommendations(self, health_status, cutting_percent, natural_damage_percent, distribution, healthy_percent):
        """Generate enhanced actionable recommendations"""
        recommendations = []
        
        # Critical cutting damage
        if cutting_percent > 35:
            recommendations.extend([
                "🚨 EMERGENCY: Report massive illegal cutting to environmental police immediately",
                "📞 Contact forest department and environmental NGOs",
                "📸 Document with photos and GPS for legal evidence",
                "🚫 Establish immediate protection zone with barriers",
                "👮 Request immediate patrol and surveillance"
            ])
        elif cutting_percent > 20:
            recommendations.extend([
                "🚨 URGENT: Report illegal cutting to forest authorities immediately",
                "🔒 Implement immediate protection measures and patrolling",
                "📋 Document damage with GPS coordinates for legal action",
                "🌱 Begin emergency replanting with native species"
            ])
        elif cutting_percent > 10:
            recommendations.extend([
                "⚠️ Investigate cutting activity and strengthen monitoring",
                "🛡️ Increase community-based protection efforts"
            ])
        
        # Recovery recommendations based on damage level
        if healthy_percent < 5 and cutting_percent > 15:
            recommendations.extend([
                "🌱 URGENT RESTORATION: Begin immediate replanting program",
                "💧 Restore water flow and tidal connectivity",
                "🧪 Test soil conditions for replanting suitability",
                "👥 Engage local communities in restoration efforts"
            ])
        
        # Natural damage responses
        if natural_damage_percent > 25:
            recommendations.extend([
                "🔬 Conduct detailed ecological assessment for disease/pest management",
                "🌊 Check water quality and salinity levels",
                "🌱 Consider assisted regeneration programs"
            ])
        
        # Healthy ecosystem maintenance
        if health_status == "HEALTHY":
            recommendations.extend([
                "✅ Continue current conservation practices",
                "📊 Maintain regular monitoring schedule",
                "🌿 Document as reference site for restoration elsewhere"
            ])
        elif distribution < 50 and healthy_percent > 10:
            recommendations.append("🌳 Focus on increasing canopy connectivity through replanting")
        
        if not recommendations:
            recommendations.append("📈 Continue monitoring and maintain current protection level")
        
        return recommendations
    
    def estimate_enhanced_recovery_time(self, cutting_percent, natural_damage_percent, health_status, healthy_percent):
        """Enhanced recovery timeline estimation"""
        if health_status == "HEALTHY":
            return "No recovery needed - maintain current state"
        elif health_status == "CLEAR_CUT" or cutting_percent > 35:
            return "20-40 years for full ecosystem recovery with intensive restoration"
        elif health_status == "SEVERELY_CUT" or cutting_percent > 25:
            return "15-25 years with active restoration and protection"
        elif cutting_percent > 15:
            return "10-15 years with protection and natural regeneration"
        elif natural_damage_percent > 30:
            return "5-10 years depending on cause and intervention"
        elif healthy_percent < 10:
            return "8-12 years with assisted regeneration"
        else:
            return "3-7 years with proper management"


class TwoStageMangroveAPI:
    """Enhanced two-stage mangrove analysis system"""
    
    def __init__(self):
        self.classifier = MangroveClassifier()
        self.damage_detector = EnhancedMangroveDamageDetector()
    
    def process_image(self, image_path):
        """Enhanced two-stage processing pipeline"""
        
        print("🔍 Stage 1: Checking if image contains mangroves...")
        
        # Extract GPS if available
        gps_coords, gps_message = extract_gps_from_image(image_path)
        
        # Stage 1: Classify if image is mangrove
        is_mangrove, classification_result = self.classifier.classify_mangrove(image_path)
        
        if not is_mangrove:
            # Special check for heavily damaged mangrove areas
            if classification_result.get("likely_damaged", False):
                print("🔄 Detected potentially damaged mangrove area, proceeding with analysis...")
                is_mangrove = True  # Override for damaged areas
                classification_result["is_mangrove"] = True
                classification_result["confidence"] = 60  # Assign moderate confidence
                classification_result["damage_override"] = True
            else:
                return {
                    "success": False,
                    "stage": 1,
                    "message": f"❌ Not a mangrove image (confidence: {classification_result['confidence']}%)",
                    "classification": classification_result,
                    "gps_location": gps_coords,
                    "suggestion": "This may be a different ecosystem or the image quality may be affecting detection."
                }
        
        print(f"✅ Stage 1 Complete: Mangrove detected (confidence: {classification_result['confidence']}%)")
        print("🔬 Stage 2: Conducting comprehensive health analysis...")
        
        # Stage 2: Enhanced damage analysis
        health_analysis = self.damage_detector.analyze_comprehensive_health(image_path)
        
        if "error" in health_analysis:
            return {"success": False, "message": health_analysis["error"]}
        
        return {
            "success": True,
            "stage": 2,
            "classification": classification_result,
            "health_analysis": health_analysis,
            "gps_location": gps_coords,
            "message": f"✅ Complete analysis finished. Health status: {health_analysis['overall_health']['status']}"
        }

def display_comprehensive_results(result):
    """Display results in user-friendly format"""
    if not result["success"]:
        print(f"\n❌ Analysis Failed: {result['message']}")
        return
    
    health = result["health_analysis"]
    
    print("\n" + "="*60)
    print("🌿 COMPREHENSIVE MANGROVE HEALTH REPORT")
    print("="*60)
    
    # GPS Location
    if result.get("gps_location"):
        lat, lng = result["gps_location"]
        print(f"📍 Location: {lat:.6f}, {lng:.6f}")
    
    # Overall Health Status
    print(f"\n🏥 OVERALL HEALTH STATUS")
    print(f"Status: {health['overall_health']['status']}")
    print(f"Health Score: {health['overall_health']['health_score']}/100")
    print(f"Description: {health['overall_health']['description']}")
    
    # Vegetation Analysis
    print(f"\n🌱 VEGETATION ANALYSIS")
    print(f"Healthy Vegetation Coverage: {health['vegetation_analysis']['healthy_vegetation']}%")
    print(f"Total Vegetation Coverage: {health['vegetation_analysis']['total_vegetation_cover']}%")
    print(f"Canopy Distribution: {health['vegetation_analysis']['canopy_distribution']}%")
    
    # Damage Assessment
    print(f"\n⚠️ DAMAGE ASSESSMENT")
    print(f"Cutting/Human Damage: {health['damage_assessment']['cutting_damage']}%")
    print(f"Natural Damage: {health['damage_assessment']['natural_damage']}%")
    print(f"Total Damage Score: {health['damage_assessment']['total_damage_score']}%")
    
    if health['damage_assessment']['primary_damage_types']:
        print("Primary Damage Types:")
        for damage_type in health['damage_assessment']['primary_damage_types']:
            print(f"  • {damage_type}")
    else:
        print("✅ No significant damage detected")
    
    # Area Estimates
    print(f"\n📏 AREA ESTIMATES")
    print(f"Healthy Area: {health['area_estimates']['estimated_healthy_area_hectares']} hectares")
    print(f"Damaged Area: {health['area_estimates']['estimated_damaged_area_hectares']} hectares")
    
    # Conservation Status
    print(f"\n🛡️ CONSERVATION STATUS")
    conservation = health['conservation_status']
    
    if conservation['requires_immediate_action']:
        print("🚨 REQUIRES IMMEDIATE ACTION")
    if conservation['alert_authorities']:
        print("📞 ALERT AUTHORITIES RECOMMENDED")
    if conservation['monitoring_required']:
        print("👁️ ENHANCED MONITORING REQUIRED")
    
    print(f"Recovery Timeline: {conservation['recovery_timeline']}")
    
    # Recommendations
    print(f"\n💡 RECOMMENDATIONS")
    for i, rec in enumerate(health['recommendations'], 1):
        print(f"{i}. {rec}")
    
    # Summary Risk Assessment
    print(f"\n📊 RISK ASSESSMENT SUMMARY")
    if health['overall_health']['health_score'] > 80:
        print("🟢 LOW RISK - Ecosystem is thriving")
    elif health['overall_health']['health_score'] > 60:
        print("🟡 MODERATE RISK - Monitor closely")
    elif health['overall_health']['health_score'] > 40:
        print("🟠 HIGH RISK - Intervention needed")
    else:
        print("🔴 CRITICAL RISK - Immediate action required")
    
    print("="*60)

def enhanced_quick_test(image_path):
    """Enhanced test function with comprehensive output"""
    
    api = TwoStageMangroveAPI()
    result = api.process_image(image_path)
    safe_result = convert_numpy_types(result)
    
    # Display comprehensive results
    display_comprehensive_results(safe_result)
    
    return safe_result

# Example usage with multiple test cases
def run_comprehensive_analysis():
    """Run analysis and demonstrate different scenarios"""
    
    # Replace with your actual image path
    test_image = "H:\\Hackout\\Mangrove\\mang.jpg"

    print("🔬 Starting Comprehensive Mangrove Analysis...")
    result = enhanced_quick_test(test_image)
    
    # Generate summary report
    if result["success"]:
        health_data = result["health_analysis"]
        
        print(f"\n📋 EXECUTIVE SUMMARY")
        print(f"Analysis Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Image Status: Confirmed Mangrove Ecosystem")
        print(f"Health Score: {health_data['overall_health']['health_score']}/100")
        print(f"Primary Concern: {health_data['damage_assessment']['primary_damage_types'][0] if health_data['damage_assessment']['primary_damage_types'] else 'None'}")
        print(f"Action Required: {'Yes' if health_data['conservation_status']['requires_immediate_action'] else 'No'}")
    
    return result

if __name__ == "__main__":
    result = run_comprehensive_analysis()

🔬 Starting Comprehensive Mangrove Analysis...
🔍 Stage 1: Checking if image contains mangroves...
✅ Stage 1 Complete: Mangrove detected (confidence: 80%)
🔬 Stage 2: Conducting comprehensive health analysis...

🌿 COMPREHENSIVE MANGROVE HEALTH REPORT

🏥 OVERALL HEALTH STATUS
Status: MODERATELY_HEALTHY
Health Score: 69.8/100
Description: Generally healthy but showing some stress signs (Score: 69.8446073130682/100)

🌱 VEGETATION ANALYSIS
Healthy Vegetation Coverage: 29.2%
Total Vegetation Coverage: 35.2%
Canopy Distribution: 90.0%

⚠️ DAMAGE ASSESSMENT
Cutting/Human Damage: 1.8%
Natural Damage: 6.3%
Total Damage Score: 8.1%
✅ No significant damage detected

📏 AREA ESTIMATES
Healthy Area: 0.292 hectares
Damaged Area: 0.081 hectares

🛡️ CONSERVATION STATUS
👁️ ENHANCED MONITORING REQUIRED
Recovery Timeline: 3-7 years with proper management

💡 RECOMMENDATIONS
1. 📈 Continue monitoring and maintain current protection level

📊 RISK ASSESSMENT SUMMARY
🟡 MODERATE RISK - Monitor closely

📋 EXECUTIVE 